In [1]:
# iterating over pages and count of articles works 
# Total articles scraped: 759
#successfully saves all links to csv

import requests
from bs4 import BeautifulSoup
import pandas as pd
import concurrent.futures
import re
import numpy as np

# Define the base URL
base_url = "https://www.sharedmobility.news/category/mob/"
page_number = 1
article_count = 1

#store the data in lists
article_texts = []
article_urls = []
article_titles = []

link_limit = 1000
while article_count <= link_limit:
    # Construct the URL for the current page
    url = f"{base_url}page/{page_number}/" if page_number > 1 else base_url

    response = requests.get(url)

    if response.status_code == 200:
        # Parse the HTML content
        html_content = response.text
        soup = BeautifulSoup(html_content, 'html.parser')

        # Finding all the links within the 'a' element within the main section
        link_elements = soup.find('main', class_='site-main rbc-content').find_all('a', class_='p-url')

        for link_element in link_elements:
            link = link_element.get('href')


            article_response = requests.get(link)
            if article_response.status_code == 200:
                article_html_content = article_response.text
                article_soup = BeautifulSoup(article_html_content, 'html.parser')

                #get article title and text
                article_title = article_soup.find('h1', class_='entry-title').get_text()
                article_text = article_soup.find('div', class_='entry-content clearfix').get_text()

                #add data to lists
                article_titles.append(article_title)
                article_texts.append(article_text)
                article_urls.append(link)
                #print("Article Title:", article_title)



                article_count += 1
                #print(f"Link {article_count}:", link)

        page_number += 1  #move to the next page
    else:
        print("Failed to retrieve the page. Status code:", response.status_code)
        break

data = {
    "ID": range(1, article_count),
    "Text": article_texts,
    "URL": article_urls,
    "Article Title": article_titles
}

# saving to a xlsx file as a csv file was breaking after 165 articles
df = pd.DataFrame(data)
df.to_excel("ride_sharing_data_full_excel.xlsx", index=False)
print(f"Total articles scraped: {article_count}")

/Users/ellasartory/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Failed to retrieve the page. Status code: 404
Total articles scraped: 760


Variance of the merged data from Opendata.nl: ID               48006.666667
Text                      NaN
URL                       NaN
Article Title             NaN
dtype: float64
ID               380.0
Text               NaN
URL                NaN
Article Title      NaN
dtype: float64


/Users/ellasartory/Library/Python/3.9/lib/python/site-packages/numpy/core/fromnumeric.py:3785: FutureWarning: The behavior of DataFrame.var with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return var(axis=axis, dtype=dtype, out=out, ddof=ddof, **kwargs)
